# Detrend and clean dataset

This notebook is used to detrend and clean dataset using the maintenance dates

In [ ]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt

### Read input file of stations

In [ ]:
ds = xr.open_dataset("../data/raw_daily_CWU.nc").sel(time=slice("2010-01-01", "2024-01-01"))
ds = ds.dropna(dim="station", how="all", subset=['e'])
ds

### Read maintenance log file in pandas

In [ ]:
def date_parser(date_str):
    return pd.to_datetime(date_str, format='%y%b%d')

df = pd.read_csv("../data/maintenance_stations.txt", delim_whitespace=True, header=None, nrows=15564, parse_dates=['date'],
            date_parser=date_parser, names = ['station', 'date', 'number', 'operation'])
df

In [ ]:
df.set_index("date", inplace=True)

# detrend with maintenance date

In [ ]:
def segment_data_by_maintenance(da, maintenance_dates, last_date="2024-01-01"):
    segments = []
    prev_date = da['time'][0]
    for date in maintenance_dates:
        segment = da.sel({'time': slice(prev_date, date)})
        if len(segment)>2:
            segments.append(segment)
            try:
                prev_date = da['time'][da['time'] > date][0]
            except:
                prev_date = -1
                # print(f"error with next segment date for {segment}")
    #check final segment
    if prev_date != -1:
        segments.append(da.sel({'time': slice(prev_date, last_date)}))  # Final segment
    return segments

def detrend_dim(da, dim="time", deg=1):
    # detrend along a single dimension
    p = da.polyfit(dim=dim, deg=deg)
    fit = xr.polyval(da[dim], p.polyfit_coefficients)
    return da - fit, fit

In [ ]:
def detrend_data_for_all_stations(data_array, df, dim="time", deg=1):
    detrended_data = []
    trends = []
    time = pd.date_range(start=data_array.time[0].item(), end=data_array.time[-1].item(), freq='1D')

    for station in data_array["station"].data:
        # try:
            station_data = data_array.sel(station=station)

            if station in df["station"].values:
                maintenance_dates = df[df["station"] == station].index
                segments = segment_data_by_maintenance(station_data, maintenance_dates)
            else:
                # No maintenance dates for this station, consider the whole dataset as one segment
                segments = [station_data]

            detrended_station_segments = []
            trend_station_segments = []
            for segment in segments:
                if len(segment) > 0:  # Check if the segment has data
                    detrended, trend = detrend_dim(segment, dim=dim, deg=deg)
                    detrended_station_segments.append(detrended)
                    trend_station_segments.append(trend)

            if len(detrended_station_segments) > 0:  # Check if there's valid detrended data
                # Only concatenate if there's data to concatenate
                detrended_da = xr.concat(detrended_station_segments, dim=dim).reindex(time=time)
                trend_da = xr.concat(trend_station_segments, dim=dim).reindex(time=time)
                detrended_data.append(detrended_da)
                trends.append(trend_da)
        # except:
        #     print(f"error with station {station}")

    # Concatenate all detrended data and trends into a single data array
    if len(detrended_data) > 0:
        detrended_combined = xr.concat(detrended_data, dim="station")
        trend_combined = xr.concat(trends, dim="station")
    else:
        detrended_combined = xr.DataArray([], dims=[dim, "station"])
        trend_combined = xr.DataArray([], dims=[dim, "station"])

    return detrended_combined, trend_combined

In [ ]:
n_detrend, trends_n = detrend_data_for_all_stations(ds['n'], df)
e_detrend, trends_e = detrend_data_for_all_stations(ds['e'], df)
z_detrend, trends_z = detrend_data_for_all_stations(ds['z'], df)

In [ ]:
dims = ('time', 'station')
ds = ds.assign(variables={"n_detrend": (dims, n_detrend.data.T)})
ds = ds.assign(variables={"e_detrend": (dims, e_detrend.data.T)})
ds = ds.assign(variables={"z_detrend": (dims, z_detrend.data.T)})
ds

## Verification at one station

In [ ]:
e_detrend.sel(station="ALBH").plot()

## Replace outliers and min max normalization

In [ ]:
def replace_outliers(data, perc=0.99):
    # calculate percentile
    threshold = data.quantile(perc)
    print(f"threshold = {threshold.data}")
    # find outliers and replace them with nan
    return data.where(abs(data)<=threshold)

In [ ]:
ds = ds.assign(variables={"n_clean": (dims, replace_outliers(ds.n_detrend).data)})
ds = ds.assign(variables={"e_clean": (dims, replace_outliers(ds.e_detrend).data)})
ds = ds.assign(variables={"z_clean": (dims, replace_outliers(ds.z_detrend).data)})

ds = ds.assign(variables={"n_norm": (dims, ((ds.n_clean.data-ds.n_clean.min().data)/(ds.n_clean.max().data-ds.n_clean.min().data))*2-1)})
ds = ds.assign(variables={"e_norm": (dims, ((ds.e_clean.data-ds.e_clean.min().data)/(ds.e_clean.max().data-ds.e_clean.min().data))*2-1)})
ds = ds.assign(variables={"z_norm": (dims, ((ds.z_clean.data-ds.z_clean.min().data)/(ds.z_clean.max().data-ds.z_clean.min().data))*2-1)})
ds

In [ ]:
import joblib

scalers = {'n':[ds.n_clean.min().data, ds.n_clean.max().data], 
           'e':[ds.e_clean.min().data, ds.e_clean.max().data], 
           'z':[ds.z_clean.min().data, ds.z_clean.max().data]}


joblib.dump(scalers, "./scalers_daily_CWU")

## Verrification at one station

In [ ]:
ds.sel(station='ALBH').e_norm.plot()

# save new ds

In [ ]:
ds.to_netcdf("../data/clean_daily_pred_CWU.nc")

In [ ]:
ds_fit = xr.Dataset(
            data_vars=dict(
                n=(["station", "time"], trends_n.data),
                e=(["station", "time"], trends_e.data),
                z=(["station", "time"], trends_z.data),
                ),
            coords=dict(
                station=(["station"], trends_n.station.data),
                time=trends_n.time.data)
            )
ds_fit

In [ ]:
ds_fit.to_netcdf("./fit_daily_CWU.nc")